# Basics of image processing with python

## Lesson 4

In this lesson, we are looking into the influence of noise and how to apply filters to remove the noise. Also, we will investigate edge detection algorithms.

### 4.1 Artificially adding random noise

In general, it is of course desirable to work with images with as little noise as possible. However, for learnning purposes, it can be helpful to manipulate images in a way to make them appear as if the sensor had some defects and the images were very noisy. This way, we can test e.g. filtering algorithms and immmediately investigate their effect on the images.

To simulate noise, we can e.g. assign random values to random pixels. Another posibility could be e.g. changing the intensity of selected pixels by a random offset or scaling actor. A lot of reserach has been done on noise models, but for this course, let's keep it simple and simply replace pixels with random values. Let's assume we would like to manipulate the greyscale image `myImg_greyscale` (It works analogously with color images, however we then have to first split the image in the three color channels and then individually manipulate each of the channels). First, we choose the coordinates of 100 random pixels (you can of course also change this factor to make the image more or less noisy and see how this affects the blurring). To select random numbers, we first have to import the `random ` library and add the following statement to the imports:

In [ ]:
import random

Next, we first pecify how many pixels should be manipulated by defining `amountDeadPixels`, and subsequenty select the random pixels by randomly choosing k=`amountDeadPixels` x- and y coordinates using `random.choice`:

In [ ]:
amountDeadPixels = 100

pixels_y_coordinates = random.choices(np.arange(0, myImg_greyscale.shape[0]), k=amountDeadPixels)
pixels_x_coordinates = random.choices(np.arange(0, myImg_greyscale.shape[1]), k=amountDeadPixels)

Now, we can assign random values in the range of 0 to 255 (assuming that our input image is 8-bit encoded) to each of the pixels of our input image, again using `random.choice` and passing the value range as arguments. In this case, the input image is directly manipulated. We could also define a new image as a copy of the input image and work on this one, in that way the input image would remain unchanged.

In [ ]:
for i in range(amountDeadPixels):
    myImg_greyscale[pixels_y_coordinates[i], pixels_x_coordinates[i]] = random.choice(np.arange(0, 255))

### 4.2 Applying convolutional filters using cv2

#### 4.2.1 General convolution

Opencv offers an easy and general way to apply any kind of 2D-convolutional filter to an image. First, you need to specify the kernel. Let us take a rectangular 3x3-sized filter as an example. The kernel `myKernel` is initialized as a 3x3 array filled with ones and subsequently divided by 9 (the sum of the elements in the array). We then create a new image `myImg_geyscale_filtered` by using `cv2.filter2D` and applying the filter kernel to the input image `myImg_geyscale`. The arguments are:
* `src` : the source image we want to manipulate
* `ddepth` : the depth of the output image. Using -1 will result in the same output depth as the input image
* `kernel` : the kernel that shall be applied to the image. We use the 

In [ ]:
myKernel = np.ones((3, 3),np.float32)/9
myImg_greyscale_filtered = cv2.filter2D(src=myImg_greyscale, ddepth=-1,kernel=myKernel)

#### 4.2.2 Pre-defined filters

A lot of filters are already implemented in cv2 and we don't have to specify the filter kernel ourselves. This is convenient for more complex and/or large filter kernels.

For example, to apply Gaussian Blur, we can use `cv2.GaussianBlur`. The arguments are:
* `src` : the source image to apply the kernel to
* `ksize` : the x- and y-dimenesions of the filter kernel (here: 9x9)
* `sigma` : the standard deviation of the Gaussian kernel (here: 3)

More arguments can be passed, please check the documentation if you are interested.

In [ ]:
myImg_greyscale_GaussianBlur = cv2.GaussianBlur(src=myImg_greyscale, ksize=(9, 9), sigma=3)

Another helpful pre-defined filter is the median filter, that can be called with `cv2.medianBlur` and accepts the following arguments:
* `src`: the source image to apply the filtering to
* `ksize` : the size of the neighborhood that should be considered to determine the median. Has to be an odd number > 1 (here:5)


In [ ]:
myImg_greyscale_MedianBlur = cv2.medianBlur(src=myImg_greyscale, ksize=5)

The bilateral filter for especially high quality edge preservation is pre-implemented and can be called with `cv2.bilateralFilter` with the following arguments:
* `src` : the source image to apply the filtering to
* `d` : Diameter of the pixel neighborhood to be considered for the filtering (here:15)
* `sigmaColor` : Filter sigma in the color space. The higher the value, the more differnet colors will be mixed into an area of equal color (here: 80)
* `sigmaSpace` : Filter sigma in the coordinate space. The higher the value, the more far away pixels will influenece the pixel result (here:80)

In [ ]:
myImg_greyscale_BilateralFilter =  cv2.bilateralFilter(src=myImg_greyscale, d=15, sigmaColor=80, sigmaSpace=80)

This is just a small selection of available filters in opencv. For more options, please check the documentation: https://docs.opencv.org/4.x/d4/d86/group__imgproc__filter.html#filter_depths

### 4.3 Edge detection

For edge detection, we can also use pre-implemented filters in opencv. 

Let's assume we would like to find the edges in `myImg_greyscale` with the Sobel operator. We can do so by using `cv2.Sobel`. However, we have to take care that the sobel operator is sensitive to thedirection that it is applied in, so we need to specify the x- and y-direction seperately. The arguments are the following:
* `src`: the source image to apply th efiltering to
* `ddepth` : the output image depth. Since the derivatives are very small, it does make sense to use a float format for the output depth, otherwise we will very likely not see any change in the output image. Therefore, using `CV_64F` is a good choice here.
* `dx`: order of the derivative x. If we want to compute the x-derivative, set to 1, for the y-derivative to 0.
* `dy` : order of the derivative y. If we want to compute the x-derivative, set to 0, for the y-derivative to 1.
* `ksize` : The kernel size of the Sobel operator, must be 1, 3, 5 or 7

In [ ]:
myImg_sobelx = cv2.Sobel(src=myImg_greyscale, ddepth=cv2.CV_64F, dx=1, dy=0, ksize=5)
myImg_sobely = cv2.Sobel(src=myImg_greyscale, ddepth=cv2.CV_64F, dx=0, dy=1, ksize=5)

The sobel operator is also depending on the edge polarity (dark-bright or bright-dark) and will result in positive or negative values, respectively. If we want to treat these edges equally, we can calculate the absolute values of the images:

In [ ]:
myImg_sobelx_abs = np.absolute(myImg_sobelx)
myImg_sobely_abs = np.absolute(myImg_sobely)

To save the result, it makes sense to transform the image back to an 8-bit format. Additionally, we can also scale the values to have the maximum possible contrast on the output image. We do so by dividing all the values by the maximum, then multiplying them by 255 and eventually converting the result to `np.uint8`:

In [ ]:
myImg_sobelx_abs_uint8 = np.uint8((myImg_sobelx_abs/np.max(myImg_sobelx_abs))*255)
myImg_sobely_abs_uint8 = np.uint8((myImg_sobely_abs/np.max(myImg_sobely_abs))*255)

The Prewitt operator is not pre-implemented. However, the differnece to the sobel operator are typically small. If you would like to use it anyway, yu can manually specify the kernel and use the general convolution as described above.

Canny edge detection is also pre-defined and can be easily implemented using `cv2.Canny` with the following arguments:
* `src` : source image to apply the edge detection to
* `threshold1` : lower threshold for the hysteresis, threshold for "weak" edges
* `threshold2` : upper threshold for the hysteresis, threshold for "strong" edges

In [ ]:
myImg_greyscale_Canny = cv2.Canny(src=myImg_greyscale, threshold1=5, threshold2=150)